In [1]:
from pyscf import gto, scf, cc
import numpy as np
from jax import numpy as jnp
from jax import vmap, jvp, jit
import jax
from functools import partial

a = 1.05835 # 2aB
nH = 2
atoms = ""
for i in range(nH):
    atoms += f"H {i*a:.5f} 0.00000 0.00000 \n"

mol = gto.M(atom=atoms, basis="6-31g", verbose=4)
mol.build()

mf = scf.RHF(mol)#.density_fit()
e = mf.kernel()

mycc = cc.CCSD(mf)
e = mycc.kernel()

System: uname_result(system='Linux', node='yichi-thinkpad', release='4.4.0-26100-Microsoft', version='#5074-Microsoft Fri Jan 01 08:00:00 PST 2016', machine='x86_64')  Threads 12
Python 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10) [GCC 13.3.0]
numpy 1.24.3  scipy 1.14.1  h5py 3.12.1
Date: Tue Oct 14 13:11:52 2025
PySCF version 2.8.0
PySCF path  /home/yichi/research/software/lno_pyscf
GIT HEAD (branch master) ef75f4190e4de208685670651dc6c467f72b6794

[ENV] PYSCF_EXT_PATH /home/yichi/research/software/pyscf
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 2
[INPUT] num. electrons = 2
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.00

In [ ]:
import numpy as np
np.set_printoptions(5)
# AFQMC/CCSD_PT energy: -1.096171 +/- 0.000554
# AFQMC/CCSD_PT energy: -2.192221 +/- 0.001636
# AFQMC/CCSD_PT energy: -4.384578 +/- 0.006862
# AFQMC/CCSD_PT energy: -8.769366 +/- 0.019849
# AFQMC/CCSD_PT energy: -17.538377 +/- 0.091299
# AFQMC/CCSD_PT energy: -27.405295 +/- 0.246627
nm = np.array([1,2,4,8,16,25])
ept = np.array([-1.096171,-2.192221,-4.384578,-8.769366,-17.538377,-27.405295])
ept_perm = ept/nm
print('number of H2 monomers:            ', nm)
print('AFQMC/CCSD_PT energy per monomer: ', ept_perm)

number of H2 monomers:             [ 1  2  4  8 16 25]
AFQMC/CCSD_PT energy per monomer:  [-1.09617 -1.09611 -1.09614 -1.09617 -1.09615 -1.09621]


In [2]:
from ad_afqmc.cisd_perturb import ccsd_pt
options = {'n_eql': 4,
           'n_prop_steps': 50,
            'n_ene_blocks': 20,
            'n_sr_blocks': 10,
            'n_blocks': 10,
            'n_walkers': 40,
            'seed': 2,
            'walker_type': 'rhf',
            'trial': 'cisd',
            'dt':0.005,
            'free_projection':False,
            'ad_mode':None,
            'use_gpu': False,
            }

from ad_afqmc import pyscf_interface
from ad_afqmc.cisd_perturb import ccsd_pt, sample_ccsd_pt
t1 = 5 * mycc.t1
mycc.t1 = t1
t2_0 = mycc.t2
mycc.t2 = t2_0
ccsd_pt.prep_afqmc(mycc,chol_cut=1e-7)
# pyscf_interface.prep_afqmc(mycc,chol_cut=1e-7)

#
# Preparing AFQMC calculation
# If you import pyscf cc modules and use MPI for AFQMC in the same script, finalize MPI before calling the AFQMC driver.
# Calculating Cholesky integrals
# Finished calculating Cholesky integrals
#
# Size of the correlation space:
# Number of electrons: (1, 1)
# Number of basis functions: 4
# Number of Cholesky vectors: 10
#


In [3]:
from ad_afqmc import config, mpi_jax, wavefunctions
import time
from jax import random
ham_data, ham, prop, trial, wave_data, sampler, observable, options, _ \
    = (mpi_jax._prep_afqmc(options))

# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Number of MPI ranks: 1
#
# norb: 4
# nelec: (1, 1)
#
# n_eql: 4
# n_prop_steps: 50
# n_ene_blocks: 20
# n_sr_blocks: 10
# n_blocks: 10
# n_walkers: 40
# seed: 2
# walker_type: rhf
# trial: cisd
# dt: 0.005
# free_projection: False
# use_gpu: False
# n_exp_terms: 6
# orbital_rotation: True
# do_sr: True
# symmetry: False
# save_walkers: False
# ene0: 0.0
# n_batch: 1
# LNO: False
# orbE: 0
# maxError: 0.001
#


In [22]:
amp_file = 'amplitudes.npz'
amplitudes = np.load(amp_file)

In [26]:
amplitudes["ci2"]

KeyError: 'ci2 is not a file in the archive'

In [7]:
def thouless_trans(t1):
    q, r = np.linalg.qr(t1)
    u_ai = r.T
    u_ji = q
    u_occ = np.vstack((u_ji,u_ai))
    u, _, _ = np.linalg.svd(u_occ)
    return u

In [ ]:
t1, t2 = wave_data['ci1'], wave_data['ci2']
u = thouless_trans(t1)
nocc = wave_data['ci1'].shape[0]
norb = trial.norb
trial_mo = u @ np.eye(norb)[:,:nocc]
wave_data['mo_coeff'] = trial_mo
# print(t1.shape,t2.shape,trial_mo.shape)
rot_t1 = trial_mo.T.conjugate()[:,:nocc] @ t1
rot_t2 = np.einsum('ij,jk,lakb->iajb',trial_mo.T.conjugate()[:,:nocc],
                   trial_mo.T.conjugate()[:,:nocc],t2)
wave_data['rot_t1'] = rot_t1
wave_data['rot_t2'] = rot_t2


(1, 3) (1, 3, 1, 3) (4, 1)


In [21]:
import h5py
chol_file='FCIDUMP_chol'
with h5py.File(chol_file, "r") as fh5:
    [nelec, nmo, ms, nchol] = fh5["header"]
    h0 = jnp.array(fh5.get("energy_core"))
    h1 = jnp.array(fh5.get("hcore")).reshape(nmo, nmo)
    h1_mod = jnp.array(fh5.get("hcore_mod")).reshape(nmo, nmo)
    chol = jnp.array(fh5.get("chol")).reshape(-1, nmo, nmo)

ham_data['h1_mod'] = h1_mod

In [ ]:
config.setup_jax()
MPI = config.setup_comm()
init = time.time()
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()
seed = options["seed"]
neql = options["n_eql"]

trial_rdm1 = trial.get_rdm1(wave_data)
if "rdm1" not in wave_data:
    wave_data["rdm1"] = trial_rdm1
ham_data = wavefunctions.rhf(
    trial.norb, trial.nelec,n_batch=trial.n_batch
    )._build_measurement_intermediates(ham_data, wave_data)
ham_data = ham.build_measurement_intermediates(ham_data, trial, wave_data)
ham_data = ham.build_propagation_intermediates(
    ham_data, prop, trial, wave_data
)
prop_data = prop.init_prop_data(trial, wave_data, ham_data, None)

e0, e1, t = ccsd_pt._ccsd_walker_energy_pt(
    prop_data['walkers'][0],ham_data,wave_data,trial)
init_ept = ham_data['h0'] + e0 + e1 - t*e0

prop_data["key"] = random.PRNGKey(seed + rank)
print('init ccsd_walker energy: ', prop_data['e_estimate'])
print('init ccsd_pt_walker energy: ', init_ept)
print('ccsd energy: ', mycc.e_tot)
eris = mycc.ao2mo(mycc.mo_coeff)
eccsd = mycc.energy(ci1, ci2, eris)
print('ccsd energy with t1*5 t2*0: ', mf.e_tot+eccsd)

# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
init ccsd_walker energy:  -1.0838115841377447
init ccsd_pt_walker energy:  -1.083811578599721
ccsd energy:  -1.1178169879811906
ccsd energy with t1*5 t2*0:  -1.0452823748261777


In [20]:
eris = mycc.ao2mo(mycc.mo_coeff)
eccsd = mycc.energy(ci1, ci2.transpose(0,2,1,3), eris)
print('ccsd energy with t1*5 t2*0: ', mf.e_tot+eccsd)
eris = mycc.ao2mo(mycc.mo_coeff)
eccsd = mycc.energy(t1, t2_0, eris)
print('ccsd energy with t1*5 t2*0: ', mf.e_tot+eccsd)

ccsd energy with t1*5 t2*0:  -1.070968514367222
ccsd energy with t1*5 t2*0:  -1.070968514367222


In [24]:
guide = wavefunctions.rhf(
    trial.norb, trial.nelec,n_batch=trial.n_batch
    )
guide._calc_energy_restricted(prop_data['walkers'][0],ham_data,wave_data)

Array(-1.07096851+0.j, dtype=complex128)

In [29]:
gf = guide._calc_green(prop_data['walkers'][0],wave_data)
print(gf.shape)

(1, 4)


In [ ]:
def _calc_overlap_restricted(self, walker: jax.Array, wave_data: dict) -> complex:
    nocc, ci1, ci2 = walker.shape[1], wave_data["ci1"], wave_data["ci2"]
    trial_wave = wave_data['mo_coeff']
    gf = (walker @ jnp.linalg.inv(trial_wave.T.conj() @ walker) @ trial_wave.T.conj()).T
    o0 = jnp.linalg.det(walker[: walker.shape[1], :]) ** 2
    o1 = jnp.einsum("ia,ia", ci1, GF[:, nocc:])
    o2 = 2 * jnp.einsum(
        "iajb, ia, jb", ci2, GF[:, nocc:], GF[:, nocc:]
    ) - jnp.einsum("iajb, ib, ja", ci2, GF[:, nocc:], GF[:, nocc:])
    return (1.0 + 2 * o1 + o2) * o0

In [109]:
from jax import lax
@jax.jit
def _hf_walker_olp(walker,wave_data):
    o0 = jnp.linalg.det(wave_data["mo_coeff"].T.conj() @ walker) ** 2
    return o0

@jax.jit
def _hf_olp_exp1(x: float,
                 h1_mod: jax.Array,
                 walker: jax.Array,
                 wave_data: dict) -> complex:
    '''
    <psi_0|exp(x*h1_mod)|walker>
    '''
    t = x * h1_mod
    walker_1x = walker + t.dot(walker)
    # olp = trial._calc_overlap_restricted(walker_1x, wave_data)
    olp = _hf_walker_olp(walker_1x,wave_data)
    return olp

@jax.jit
def _hf_olp_exp2(x: float,
                 chol_i: jax.Array, 
                 walker: jax.Array,
                 wave_data: dict) -> complex:
    '''
    <psi_0|exp(x*h2_mod)|walker>
    '''
    walker_2x = (
            walker
            + x * chol_i.dot(walker)
            + x**2 / 2.0 * chol_i.dot(chol_i.dot(walker))
        )
    olp = _hf_walker_olp(walker_2x, wave_data)
    # olp = trial._calc_overlap_restricted(walker_2x, wave_data)
    return olp

@partial(jit, static_argnums=3)
def _e0_new(walker, wave_data, ham_data, trial, eps=3e-4):
    '''<psi_0|h1+h2|phi>/<psi_0|phi>'''
    norb = trial.norb
    # h0 = ham_data['h0']
    h1_mod = ham_data['h1_mod']
    chol = ham_data["chol"].reshape(-1, norb, norb)

    # one body
    x = 0.0
    f1 = lambda a: _hf_olp_exp1(a,h1_mod,walker,wave_data)
    olp, d_olp = jvp(f1, [x], [1.0])

    # two body
    def scanned_fun(carry, c):
        eps, walker, wave_data = carry
        return carry, _hf_olp_exp2(eps,c,walker,wave_data)

    _, olp_p = lax.scan(scanned_fun, (eps, walker,wave_data), chol)
    _, olp_0 = lax.scan(scanned_fun, (0.0, walker,wave_data), chol)
    _, olp_m = lax.scan(scanned_fun, (-1.0 * eps, walker,wave_data), chol)
    d_2_olp = (olp_p - 2.0 * olp_0 + olp_m) / eps / eps

    # e_0= h0 + (d_olp + jnp.sum(d_2_olp) / 2.0 )/ olp
    e_0= (d_olp + jnp.sum(d_2_olp) / 2.0 )/ olp

    return e_0

In [ ]:
@jax.jit
def _t2_walker_olp(walker,wave_data):
    ''' t2_ij^ab <psi|t2_ij^ab|phi> '''
    t2 = wave_data['t2']
    nocc = walker.shape[1]
    GF = (walker.dot(jnp.linalg.inv(walker[: nocc, :]))).T
    o0 = _hf_walker_olp(walker,wave_data)
    o2 = 2 * jnp.einsum(
        "iajb, ia, jb", t2, GF[:, nocc:], GF[:, nocc:]
    ) - jnp.einsum("iajb, ib, ja", t2, GF[:, nocc:], GF[:, nocc:])
    return o2 * o0

@jax.jit
def _t2_olp_exp1(x: float, h1_mod: jax.Array, walker: jax.Array,
                  wave_data: dict) -> complex:
    '''
    c_ia <psi_i^a|exp(x*h1_mod)|walker>
    '''
    t = x * h1_mod
    walker_1x = walker + t.dot(walker)
    olp = _t2_walker_olp(walker_1x,wave_data)
    return olp

@jax.jit
def _t2_olp_exp2(x: float, chol_i: jax.Array, walker: jax.Array,
                  wave_data: dict) -> complex:
    '''
    c_ia <psi_i^a|exp(x*h2_mod)|walker>
    '''
    walker_2x = (
            walker
            + x * chol_i.dot(walker)
            + x**2 / 2.0 * chol_i.dot(chol_i.dot(walker))
        )
    olp = _t2_walker_olp(walker_2x,wave_data)
    return olp

@partial(jit, static_argnums=3)
def _e2(walker, ham_data, wave_data, trial, eps=3e-4):
    norb = trial.norb
    h0 = ham_data['h0']
    h1_mod = ham_data['h1_mod']
    chol = ham_data["chol"].reshape(-1, norb, norb)

   
    # zero body
    # c_ij^ab <psi_ij^ab|phi>/<psi_0|phi> * h0
    hf_olp = _hf_walker_olp(walker)
    t2_olp = _t2_walker_olp(walker,wave_data)

    # one body
    # c_ij^ab <psi_ij^ab|phi_1x>/<psi_0|phi>
    x = 0.0
    f1 = lambda a: _t2_olp_exp1(a,h1_mod,walker,wave_data)
    _, d_olp = jvp(f1, [x], [1.0])

    # two body
    # c_ij^ab <psi_ij^ab|phi_2x>/<psi_0|phi>
    def scanned_fun(carry, c):
        eps, walker, wave_data = carry
        return carry, _t2_olp_exp2(eps,c,walker,wave_data)

    _, olp_p = lax.scan(scanned_fun, (eps, walker, wave_data), chol)
    _, olp_0 = lax.scan(scanned_fun, (0.0, walker, wave_data), chol)
    _, olp_m = lax.scan(scanned_fun, (-1.0 * eps, walker, wave_data), chol)
    d_2_olp = (olp_p - 2.0 * olp_0 + olp_m) / eps / eps
    
   
    e_2 = (h0 * t2_olp + d_olp + jnp.sum(d_2_olp) / 2.0 ) / hf_olp

    return e_2

In [110]:
wave_data["mo_coeff"] = np.eye(norb)[:,:nocc]

In [123]:
from ad_afqmc.cisd_perturb import sample_pt2
prop_data["n_killed_walkers"] = 0
prop_data, (blk_wt, blk_t, blk_e0, blk_e1) = \
    sample_pt2._block_scan(prop_data,ham_data,prop,trial,wave_data,sampler)

In [124]:
_e0_new(prop_data['walkers'][0], wave_data, ham_data, trial)+h0

Array(-1.10200328+0.0001069j, dtype=complex128)

In [125]:
guide._calc_energy_restricted(prop_data['walkers'][0], ham_data, wave_data)

Array(-1.10200328+0.0001069j, dtype=complex128)

In [115]:
_e0_new(prop_data['walkers'][0], wave_data, ham_data, trial)+h0

Array(-1.10508331+0.00444493j, dtype=complex128)

In [113]:
print(blk_wt)
print(blk_t, blk_e0, blk_e1)

40.18962070410524
-0.001987533120296385 -1.5928842369972318 -0.0030159243145234703
